Set-up

In [34]:
import os
import sys
import json
import openai
from langchain.tools import tool
from pydantic.v1 import BaseModel, Field
from langchain.chat_models import ChatOpenAI
from langchain.schema.agent import AgentFinish
from langchain.prompts import ChatPromptTemplate
from langchain.prompts import MessagesPlaceholder
from langchain.agents.format_scratchpad import format_to_openai_functions
from langchain.agents.output_parsers import OpenAIFunctionsAgentOutputParser
from langchain_core.utils.function_calling import convert_to_openai_function
sys.path.append('../..')

In [9]:
# Setting your personal OPENAI Api Key
openai.api_key = os.environ['OPENAI_API_KEY']

Initializing Tools

In [10]:
ui_function_identifier = "This is a UI function"

In [44]:
class Design_logo_input(BaseModel):
    topic: str = Field(..., description="The topic for which the logo should be designed for. For example, if the topic were 'ice-cream shop' then the logo returned would be a logo fit for the ice-cream shop to put on their sign and merchandise.")

@tool(args_schema=Design_logo_input)
def design_logo_concepts(topic: str) -> str:
    """Returns the url for a logo design that is based on the provided topic"""
    # Currently a very empty function
    return f"s3://designs/logo_{len(topic)%5+1}.jpg"

In [89]:
class UI_Output_Request(BaseModel):
    inpt: str = Field(... , description=f"A description of the UI function you want and how you would call it along with the arguments you would use")

@tool(args_schema=UI_Output_Request)
def ui_output_request(inpt: str) -> str:
    """Provides a convenient way for you to describe what UI function you want and what to call it with without actually calling a UI function."""
    return inpt

In [90]:
class Product_UI_input(BaseModel):
    path: str = Field(... , description=f"The path of the image for the UI page. If no image is wanted or required provide an empty string.")

@tool(args_schema=Product_UI_input)
def produce_image_UI(path: str) -> str:
    """This is a UI function. Produces the JSON for a general UI page about an image."""
    data = {"component":"Image", "props":{"url":path}}
    return  json.dumps(data)            

Registering Tools

In [105]:
tools = [design_logo_concepts, produce_image_UI, ui_output_request]
functions = [convert_to_openai_function(f) for f in tools]

In [91]:
tool_dictionary = {
        "design_logo_concepts": design_logo_concepts,
        "produce_image_UI": produce_image_UI,
        "ui_output_request": ui_output_request
    }

In [118]:
ui_tool_dictionary = {
    "produce_image_UI": produce_image_UI
}

Setting up LLM Instructions

In [79]:
agent_system_message = "You are helpful assistant, don't be afraid to use the functions provided to you."

In [100]:
initial_message_beginning =  "You are not allowed to call UI functions (which are the functions whose descriptions contains the phrase" +\
    f" '{ui_function_identifier}') as a response to this particular message. If you feel like you need to call the a UI function" +\
    " call 'ui_output_request' instead."

In [103]:
correction_message = "No! You used a UI function which I said you weren't allowed to! Take what I write next and create a function call for" +\
    " 'ui_output_request'."

In [81]:
cognitive_ui_instructions = f"Your task is to take the information from the scratchpad and produce an output" +\
        " in the appropriate format. You have two choices, you can either produce a UI page by calling one of the UI functions provided" +\
        " to you (and returning the function call) or just output plain text like you are used to. The UI functions are functions that have" +\
        f" the phrase '{ui_function_identifier}' in their description, do not use functions that don't have this. If the appropriate" +\
        " UI function isn't available, use the closest fitting UI function, lack of appropriate function should not dissuade you" +\
        " from choosing the UI output option."

Initializing Chain + Agent

In [106]:
model = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0).bind(functions=functions)
prompt = ChatPromptTemplate.from_messages([
    ("system", agent_system_message),
    ("user", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad")
])
chain = prompt | model | OpenAIFunctionsAgentOutputParser()

In [121]:
def run_agent(user_input):
    """
    Runs the UI agent which will return a tuple of (LLM Response, Is it calling a UI Function)
    """
    intermediate_steps = []
    result = chain.invoke({
        "input": initial_message_beginning + user_input,
        "agent_scratchpad": format_to_openai_functions(intermediate_steps)
    })
    # Are we calling the functions? (If not we return)
    if isinstance(result, AgentFinish):
        return result, False
    # Check if UI tool was run pre-emptively and if, re-run
    if result.tool in ui_tool_dictionary:
        # print ("Agent was naughty")
        result = chain.invoke({
            "input": correction_message + user_input,
            "agent_scratchpad": format_to_openai_functions(intermediate_steps)
        })
    # Run first non-UI function
    tool = tool_dictionary[result.tool]
    observation = tool.run(result.tool_input)
    intermediate_steps.append((result, observation))
    # Cognitive UI Steps
    result = chain.invoke({
        "input": cognitive_ui_instructions,
        "agent_scratchpad": format_to_openai_functions(intermediate_steps)
    })
    tool = tool_dictionary[result.tool]
    observation = tool.run(result.tool_input)
    return observation, True

Testing

In [122]:
print(run_agent("Hello!"))

(AgentFinish(return_values={'output': 'Hello! How can I assist you today?'}, log='Hello! How can I assist you today?'), False)


In [123]:
print(run_agent("Design me a big red logo"))

('{"component": "Image", "props": {"url": "s3://designs/logo_3.jpg"}}', True)


In [124]:
print(run_agent("Tell me a bit about Cornell University, I hear they are also reffered to as Big Red."))

(AgentFinish(return_values={'output': 'Cornell University is a private Ivy League research university located in Ithaca, New York. It was founded in 1865 and is widely regarded as one of the top universities in the world. Cornell is known for its diverse academic programs, including colleges of Agriculture and Life Sciences, Arts and Sciences, Engineering, and more.\n\nThe nickname "Big Red" refers to Cornell\'s athletic teams, which compete in the NCAA Division I Ivy League conference. The university\'s sports teams are known for their competitive spirit and have a strong following among students and alumni.\n\nCornell University has a beautiful campus with stunning natural surroundings, including gorges and waterfalls. It offers a vibrant and intellectually stimulating environment for students, with numerous research opportunities and a strong emphasis on interdisciplinary studies.\n\nOverall, Cornell University is renowned for its academic excellence, research contributions, and vib

In [125]:
print(run_agent("Make me a nice UI page about the movie 'Dune'"))

('{"component": "Image", "props": {"url": "Dune"}}', True)
